In [48]:
ann_dir='/home/all/datasets/caltech/annotations'
image_identifiers = get_image_identifiers('/home/xuzhewei/code/py-faster-rcnn-caltech-pedestrian/data/caltech/data/ImageSets/train_1x.txt')
recs = get_caltech_annoations(image_identifiers, '/home/all/datasets/caltech/annotations')
recs_filted = get_caltech_annoations(image_identifiers, '/home/all/datasets/caltech/annotations',pLoad)

In [29]:
bboxs = list(recs['set00/V001/1'])
print recs['set00/V001/1']
print bboxs

[{'lock': 0, 'pos': [356.3214716525935, 95.69581825492567, 35.023723361479696, 28.667470848411742], 'lbl': 'people', 'ignore': 0, 'occl': 0, 'posv': [0, 0, 0, 0], 'id': 2}, {'lock': 0, 'pos': [515.3434593219138, 167.64628790429808, 20.254186389581264, 42.082034942680224], 'lbl': 'person', 'ignore': 0, 'occl': 1, 'posv': [518.9488840098322, 169.94383400945642, 14.67895163762455, 21.34106402799509], 'id': 1}, {'lock': 0, 'pos': [466.82235152203225, 164.15904822725236, 26.059604058752484, 51.879282859102574], 'lbl': 'person', 'ignore': 0, 'occl': 0, 'posv': [0, 0, 0, 0], 'id': 0}]
[{'lock': 0, 'pos': [356.3214716525935, 95.69581825492567, 35.023723361479696, 28.667470848411742], 'lbl': 'people', 'ignore': 0, 'occl': 0, 'posv': [0, 0, 0, 0], 'id': 2}, {'lock': 0, 'pos': [515.3434593219138, 167.64628790429808, 20.254186389581264, 42.082034942680224], 'lbl': 'person', 'ignore': 0, 'occl': 1, 'posv': [518.9488840098322, 169.94383400945642, 14.67895163762455, 21.34106402799509], 'id': 1}, {'lo

In [11]:
pLoad=get_default_filter()
pLoad['lbls']=['person']
pLoad['ilbls']=['people']
pLoad['squarify']=[3,0.41]
pLoad['hRng']=[50,float('inf')]
pLoad['vRng']=[1,1]

In [30]:
bbox_filter(bboxs,pLoad)

/home/all/lib/anaconda2/envs/caffe/lib/python2.7/site-packages/ipykernel/__main__.py:19: DeprecationWarning: `Tracer` is deprecated since version 5.1, directly use `IPython.core.debugger.Pdb.set_trace()`


> <ipython-input-10-f078d518179a>(20)bbox_filter()
     18     # param = get_param_default(param,df)
     19     Tracer()()
---> 20     n = len(bboxs)
     21     bbox_filted = []
     22 

ipdb> q
Exiting Debugger.


In [62]:
image_identifier = 'set00/V001/31'
print recs[image_identifier]
print '-'*10
print recs_filted[image_identifier]

[{'lock': 0, 'pos': [360.64475271411345, 84.16706875753925, 38.98673100120621, 30.10856453558503], 'lbl': 'people', 'ignore': 0, 'occl': 0, 'posv': [0, 0, 0, 0], 'id': 2}, {'lock': 0, 'pos': [592.9438320171336, 162.7011595322414, 26.38192933131961, 66.9459701141121], 'lbl': 'person', 'ignore': 0, 'occl': 0, 'posv': [0, 0, 0, 0], 'id': 1}, {'lock': 0, 'pos': [524.6294101765804, 159.21427304335495, 22.97899666501121, 66.7635992659792], 'lbl': 'person', 'ignore': 0, 'occl': 0, 'posv': [0, 0, 0, 0], 'id': 0}]
----------
[{'lock': 0, 'pos': [360.64475271411345, 84.16706875753925, 38.98673100120621, 30.10856453558503], 'lbl': 'people', 'ignore': True, 'occl': 0, 'posv': [0, 0, 0, 0], 'id': 2}, {'lock': 0, 'pos': [592.4108728094004, 162.7011595322414, 27.44784774678596, 66.9459701141121], 'lbl': 'person', 'ignore': False, 'occl': 0, 'posv': [0, 0, 0, 0], 'id': 1}, {'lock': 0, 'pos': [522.4323706595603, 159.21427304335495, 27.37307569905147, 66.7635992659792], 'lbl': 'person', 'ignore': False,

In [34]:
from scipy.io import loadmat
from collections import defaultdict
import json
import glob
import os
import math
from IPython.core.debugger import Tracer

In [35]:
def load_vbb(filename):
    '''
    A is a dict load from the caltech vbb file has the same data structure.
    '''
    vbb = loadmat(filename)
    nFrame = int(vbb['A'][0][0][0][0][0])
    objLists = vbb['A'][0][0][1][0]
    maxObj = int(vbb['A'][0][0][2][0][0])
    objInit = vbb['A'][0][0][3][0]
    objLbl = [str(v[0]) for v in vbb['A'][0][0][4][0]]
    objStr = vbb['A'][0][0][5][0]
    objEnd = vbb['A'][0][0][6][0]
    objHide = vbb['A'][0][0][7][0]
    altered = int(vbb['A'][0][0][8][0][0])
    log = vbb['A'][0][0][9][0]
    logLen = int(vbb['A'][0][0][10][0][0])
    
    obj_list = defaultdict(dict)
    for frame_id, obj in enumerate(objLists):
        objs = []
        if len(obj) > 0:
            for id, pos, occl, lock, posv in zip(obj['id'][0], obj['pos'][0], obj['occl'][0], obj['lock'][0], obj['posv'][0]):
                id = int(id[0][0])-1 # matlab is 1-start
                pos = pos[0].tolist()
                occl = int(occl[0][0])
                lock = int(lock[0][0])
                posv = posv[0].tolist()
                keys_obj = ('id','pos','occl','lock','posv','ignore')
                datum = dict(zip(keys_obj, [id, pos, occl, lock, posv, 0]))
                datum['lbl'] = objLbl[id]
                objs.append(datum)
            obj_list[frame_id+1] = objs
    
    keys_vbb = ('nFrame','objLists','maxObj','objInit','objLbl','objStr','objEnd','objHide','altered','log','logLen')
    A = dict(zip(keys_vbb, [nFrame, obj_list, maxObj, objInit, objLbl, objStr, objEnd, objHide, altered, log, logLen]))
    return A

In [36]:
def get_caltech_annoations(image_identifiers, ann_dir, param={}):
    '''
    get all image_identifiers annotation

    INPUTS:
        image_identifiers :
        ann_dir :
    OUPUT:
        anno : {'set00/V000/121': [{'id': 3,
                                    'lbl': 'person',
                                    'lock': 0,
                                    'occl': 1,
                                    'pos': [230.68536627473338,131.379092381353,7.045344709775236,13.1208549490851],
                                    'posv': [230.68536627473338,131.379092381353,7.045344709775236,13.1208549490851]}],
                                    ...
                }
    '''
    anno = {}
    data = defaultdict(dict)
    for dname in sorted(glob.glob(ann_dir+'/set*')):
        set_name = os.path.basename(dname)
        data[set_name] = defaultdict(dict)
        for anno_fn in sorted(glob.glob('{}/*.vbb'.format(dname))):
            vid_anno = load_vbb(anno_fn)
            video_name = os.path.splitext(os.path.basename(anno_fn))[0]
            data[set_name][video_name] = vid_anno['objLists']
            
    for image_identifier in image_identifiers:
        image_set_name = image_identifier[0:5]
        image_seq_name = image_identifier[6:10]
        image_id       = int(image_identifier[11:])
#         Tracer()()
        if image_id in data[image_set_name][image_seq_name]:
#             Tracer()()
            if len(param)!=0:
                #Tracer()()
                anno[image_identifier] = bbox_filter(data[image_set_name][image_seq_name][image_id],param)
            else:
                anno[image_identifier] = data[image_set_name][image_seq_name][image_id]
        else:
            print "Warning: No %s.jpg found in annotations" %(image_identifier)
           
        #vis_annotations(image_identifier, anno[image_identifier])
    return anno

In [37]:
def get_image_identifiers(imageSets_file):
    '''get image identifiers from ImageSets dir's file for train or test. like {'set00/V000/1','set/V000/31'...}'''
    with open(imageSets_file, 'r') as f:
        lines = f.readlines()
    image_identifiers = [x.strip() for x in lines]

    return image_identifiers

In [38]:
def bbox_filter(bboxs,param):
    
    #Tracer()()
    n = len(bboxs)
    bbox_filted = []
    
    if len(param['lbls']) != 0:
        lbl = set(param['lbls'])|(set(param['ilbls'])) 
        for i in range(n):            
            if bboxs[i]['lbl'] in lbl:
                bbox_filted.append(bboxs[i])
    
    for i in range(len(bbox_filted)):
        #Tracer()()
        if len(param['ilbls']) != 0:
            bbox_filted[i]['ignore'] = bbox_filted[i]['ignore'] or (bbox_filted[i]['lbl'] in param['ilbls'])
        if len(param['xRng']) != 0:
            v = bboxs[i]['pos'][0]
            bbox_filted[i]['ignore'] = bbox_filted[i]['ignore'] or v < param['xRng'][0] or v > param['xRng'][1]
            v = bbox_filted[i]['pos'][0] + bbox_filted[i]['pos'][2]
            bbox_filted[i]['ignore'] = bbox_filted[i]['ignore'] or v < param['xRng'][0] or v > param['xRng'][1]
        if len(param['yRng']) != 0:
            v = bbox_filted[i]['pos'][1]
            bbox_filted[i]['ignore'] = bbox_filted[i]['ignore'] or v < param['yRng'][0] or v > param['yRng'][1]
            v = bbox_filted[i]['pos'][1] + bbox_filted[i]['pos'][3]
            bbox_filted[i]['ignore'] = bbox_filted[i]['ignore'] or v < param['yRng'][0] or v > param['yRng'][1]
        if len(param['wRng']) != 0:
            v = bbox_filted[i]['pos'][2]
            bbox_filted[i]['ignore'] = bbox_filted[i]['ignore'] or v < param['wRng'][0] or v > param['wRng'][1]
        if len(param['hRng']) != 0:
            v = bbox_filted[i]['pos'][3]
            bbox_filted[i]['ignore'] = bbox_filted[i]['ignore'] or v < param['hRng'][0] or v > param['hRng'][1]
        if len(param['aRng']) != 0:
            v = bbox_filted[i]['pos'][2] * bbox_filted[i]['pos'][3]
            bbox_filted[i]['ignore'] = bbox_filted[i]['ignore'] or v < param['aRng'][0] or v > param['aRng'][1]
        if len(param['arRng']) != 0:
            v = bbox_filted[i]['pos'][2] / bbox_filted[i]['pos'][3]
            bbox_filted[i]['ignore'] = bbox_filted[i]['ignore'] or v < param['arRng'][0] or v > param['arRng'][1]
        if len(param['vRng']) != 0:
            pos  = bbox_filted[i]['pos']
            posv = bbox_filted[i]['posv']
            if bbox_filted[i]['occl']==0 or set(posv)=={0}:
                v = 1
            elif cmp(posv,pos):
                v = 0
            else:
                v = (posv[2]*posv[3])/(pos[2]*pos[3])
            bbox_filted[i]['ignore'] = bbox_filted[i]['ignore'] or v < param['vRng'][0] or v > param['vRng'][1]
        if len(param['occl']) != 0:
            '''occl= 0|1|2 represent for no occl | occl | no and occl'''
            if param['occl'] != 2:
                bbox_filted[i]['ignore'] = bbox_filted[i]['ignore'] or (bbox_filted[i]['occl'] != param['occl'])
        if (bbox_filted[i]['ignore'] == 0) and (len(param['squarify']) != 0):
            bbox_filted[i]['pos'] = bbox_squarify(bbox_filted[i]['pos'],param['squarify'][0],param['squarify'][1])
        #Tracer()()
    #Tracer()()
    return bbox_filted

In [39]:
def get_default_filter():
    df = dict()
    df['format']  = 0
    df['ellipse'] = 1
    df['squarify']= []
    df['lbls']    = []
    df['ilbls']   = []
    df['hRng']    = []
    df['wRng']    = []
    df['aRng']    = []
    df['arRng']   = []
    df['oRng']    = []
    df['xRng']    = []
    df['yRng']    = []
    df['vRng']    = []
    df['occl']    = []
    df['squarify'] = []
    
    return df

In [40]:
def is_empty(input_object):
    return (input_object is None) or (len(input_object) == 0)

In [41]:
def get_image_identifiers(imageSets_file):
    
    with open(imageSets_file, 'r') as f:
        lines = f.readlines()
    image_identifiers = [x.strip() for x in lines]

    return image_identifiers

In [42]:
def bbox_squarify(bb,flag,ar=1):

    bbr=bb
    if flag == 4:
        bbr = bbox_resize(bb,0,0,ar)
        return bbr
    usew = (flag == 0 and (bb[2]>bb[3]*ar)) or (flag == 1 and (bb[2]<bb[3]*ar)) or flag == 2
    if usew:
        bbr = bbox_resize(bb,0,1,ar)
    else:
        bbr = bbox_resize(bb,1,0,ar)
        
    return bbr

In [43]:
def bbox_resize(bb,hr,wr,ar=0):
    assert len(bb)==4
    assert (hr>0 and wr>0) or ar>0
    if hr==0 and wr==0:
        a = math.sqrt(bb[2]*bb[3])
        ar= math.sqrt(ar)
        d = a*ar - bb[2]; bb[0]=bb[0]-d/2; bb[2]=bb[2]+d
        d = a*ar - bb[3]; bb[1]=bb[1]-d/2; bb[3]=bb[3]+d
        return bb
    if hr!=0:
        d=(hr-1)*bb[3]; bb[1]=bb[1]-d/2; bb[3]=bb[3]+d
    if wr!=0:
        d=(hr-1)*bb[2]; bb[0]=bb[0]-d/2; bb[2]=bb[2]+d
    if hr==0:
        d=bb[2]/ar-bb[3]; bb[1]=bb[1]-d/2; bb[3]=bb[3]+d
    if wr==0:
        d=bb[3]*ar-bb[2]; bb[0]=bb[0]-d/2; bb[2]=bb[2]+d
    
    return bb